De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3n} \left [ f(x_0) + f(x_n) + 2 \sum_{i=1}^{\frac{n}{2}-1}f(x_{2i}) + 4 \sum_{i=1}^{\frac{n}{2}}f(x_{2i-1})\right ] $$

con $h=b-a$ y $n$ número de subintervalos (par).

**Nota:** Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos. Estos índices se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices que se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [5]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [6]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [9]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

In [10]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [11]:
p= #número de subprocesos a lanzar

In [12]:
n= #número de subintervalos a lanzar
h_hat=h/n

In [13]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [23]:
nt

In [24]:
ns_p

In [16]:
ns_p=int(ns_p)

In [17]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) #n always even number, that's why
                                               #we divide by 6
                                               #we are summing twice last node
                                               #so we need to substract f(b)
    end_time=time.time()

In [25]:
aprox

In [19]:
obj, err = quad(f, a, b)

In [20]:
obj

0.7468241328124271

In [21]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [26]:
err_relativo(aprox,obj)